# ResNet50 fine-tuning for classification on medical images

## Part 1. Train the model

In [1]:
# Main variables
gpu = 3  # for DL4 server GPU can be: 0 (H800), 1 (V100), 2 (V100), 3 (V100), 4 (V100).
out_dir = "./outputs_ResNet"

model_name = "wide_resnet50_2.tv2_in1k"  # ResNet50 for 224x224 images
patience = 10  # how long to wait after the last best improvement before stopping

# data_dir = "/share/data/lab225/ViT/img_CT_224_Gray_MF_c1_c2_c3"  # dataset directory
# csv_out_1   = f"{out_dir}/FTR_G2_F_c1_train_fnames.csv"  # CVS file with file names
# csv_out_2   = f"{out_dir}/FTR_G2_F_c1_train_featur.csv"  # CVS file with features

data_dir = "./xray_dataset"  # dataset directory
csv_out_1   = f"{out_dir}/FTR_path_train_fnames.csv"  # CVS file with file names
csv_out_2   = f"{out_dir}/FTR_path_train_featur.csv"  # CVS file with features

In [2]:
# Expected data layout:
# data/
#     train/
#         classA/
#         classB/
#         ….
#     val/ (optional; will be auto-split from train if missing)
#         classA/
#         classB/
#         ….
import os
import sys
import math
import time
import timm  # requires: pip install timm
import random
import argparse
import multiprocessing

from pathlib import Path

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn

from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms


def set_seed(seed: int = 42):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = False
    cudnn.benchmark = True


def build_transforms(img_size=224):
    # # previous mean and STD values
    # mean = [0.485, 0.456, 0.406]
    # std = [0.229, 0.224, 0.225]

    # new mean and STD values
    mean = [0.5, 0.5, 0.5]
    std = [0.5, 0.5, 0.5]

    train_tfms = transforms.Compose([
        transforms.Resize(int(img_size)),
        # transforms.RandomResizedCrop(img_size, scale=(0.6, 1.0)),
        # transforms.RandomHorizontalFlip(p=0.5),
        # transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])
    val_tfms = transforms.Compose([
        transforms.Resize(int(img_size)),
        # transforms.Resize(int(img_size * 1.14)),
        # transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])
    return train_tfms, val_tfms


def create_dataloaders(data_dir, batch_size, num_workers, img_size, val_split):
    train_tfms, val_tfms = build_transforms(img_size)

    train_dir = Path(data_dir) / "train"
    val_dir = Path(data_dir) / "val"

    if train_dir.exists() and val_dir.exists():
        train_ds = datasets.ImageFolder(str(train_dir), transform=train_tfms)
        val_ds = datasets.ImageFolder(str(val_dir), transform=val_tfms)
    else:
        # Single folder with class subfolders; auto-split into train/val
        base_dir = train_dir if train_dir.exists() else Path(data_dir)
        full_ds = datasets.ImageFolder(str(base_dir), transform=train_tfms)
        n_total = len(full_ds)
        n_val = int(n_total * val_split)
        n_train = n_total - n_val
        train_ds, val_ds = random_split(
            full_ds, [n_train, n_val],
            generator=torch.Generator().manual_seed(42)
        )
        # Apply val transforms to the validation subset
        val_ds.dataset = datasets.ImageFolder(
            str(base_dir),
            transform=val_tfms
        )

    train_loader = DataLoader(
        train_ds, batch_size=batch_size, shuffle=True,
        num_workers=num_workers, pin_memory=True, drop_last=True
    )
    val_loader = DataLoader(
        val_ds, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, pin_memory=True
    )
    classes = train_ds.dataset.classes \
        if hasattr(train_ds, "dataset") \
        else train_ds.classes
    num_classes = len(classes)
    return train_loader, val_loader, num_classes, classes


def build_model(
    model_name, num_classes,
    pretrained=True, freeze_backbone=False
):
    # global_pool='avg' yields pooled features (B, C) from forward_features
    model = timm.create_model(
        model_name, pretrained=pretrained,
        num_classes=num_classes, global_pool='avg'
    )

    if freeze_backbone:
        stop_words = ['head', 'fc', 'classifier']
        for name, p in model.named_parameters():
            if all(sw not in name for sw in stop_words):
                p.requires_grad = False

    return model


def accuracy(output, target, topk=(1,)):
    with torch.no_grad():
        maxk = max(topk)
        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1))
        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0)
            res.append((correct_k * (100.0 / target.size(0))).item())
        return res


def create_optimizer(model, lr, weight_decay):
    # Only update trainable parameters
    params = [p for p in model.parameters() if p.requires_grad]
    return torch.optim.AdamW(params, lr=lr, weight_decay=weight_decay)


def create_scheduler(optimizer, warmup_epochs, max_epochs, train_loader_len):
    # Linear warmup then cosine decay
    total_steps = max_epochs * train_loader_len
    warmup_steps = warmup_epochs * train_loader_len

    def lr_lambda(step):
        if step < warmup_steps:
            return float(step) / float(max(1, warmup_steps))
        # Cosine decay from 1.0 to 0.0
        progress = (step - warmup_steps) / float(max(1, total_steps - warmup_steps))
        return 0.5 * (1.0 + torch.cos(torch.tensor(progress * 3.1415926535))).item()

    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)


def train_one_epoch(
    model, loader, optimizer, scaler,
    device, criterion, log_interval=50
):
    model.train()
    running_loss = 0.0
    running_acc1 = 0.0
    n = 0

    for i, (images, targets) in enumerate(loader):
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        with torch.autocast(device, dtype=torch.float32):
            outputs = model(images)
            loss = criterion(outputs, targets)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        acc1 = accuracy(outputs, targets, topk=(1,))[0]
        bs = images.size(0)
        running_loss += loss.item() * bs
        running_acc1 += acc1 * bs
        n += bs

        if (i + 1) % log_interval == 0:
            print(f"\t[batch {i+1}/{len(loader)}] loss={running_loss/n:.4f} acc1={running_acc1/n:.2f}%")

    return running_loss / n, running_acc1 / n


@torch.no_grad()
def evaluate(model, loader, device, criterion):
    model.eval()
    running_loss = 0.0
    running_acc1 = 0.0
    n = 0
    for images, targets in loader:
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)
        with torch.autocast(device, dtype=torch.float32):
            outputs = model(images)
            loss = criterion(outputs, targets)
        acc1 = accuracy(outputs, targets, topk=(1,))[0]
        bs = images.size(0)
        running_loss += loss.item() * bs
        running_acc1 += acc1 * bs
        n += bs
    return running_loss / n, running_acc1 / n


def save_checkpoint(model, optimizer, scheduler, epoch, accuracy, loss, path):
    state = {
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
        "scheduler": scheduler.state_dict() if scheduler is not None else None,
        "epoch": epoch,
        "best_val_acc": accuracy,
        "min_val_loss": loss,
    }
    torch.save(state, path)


def train(model, device, args, train_loader, val_loader, num_classes,
          class_names, out_dir, ckpt_path, patience, freeze):
    """ Train the model. """
    criterion = nn.CrossEntropyLoss()
    optimizer = create_optimizer(model, args.lr, args.weight_decay)
    scheduler = create_scheduler(optimizer, args.warmup_epochs, args.epochs, len(train_loader))

    scaler = torch.amp.GradScaler(enabled=(device == "cuda"))

    start_epoch = 0
    no_improvement_count = 0
    best_val_acc = 0.0  # validation accuracy
    min_val_loss = float("inf")  # minimal validation loss

    if args.resume and os.path.isfile(args.resume):
        ckpt = torch.load(args.resume, map_location="cpu")
        model.load_state_dict(ckpt["model"], strict=False)
        if ckpt.get("optimizer"):
            optimizer.load_state_dict(ckpt["optimizer"])
        if ckpt.get("scheduler") and scheduler is not None and ckpt["scheduler"] is not None:
            scheduler.load_state_dict(ckpt["scheduler"])
        start_epoch = ckpt.get("epoch", 0) + 1
        best_val_acc = ckpt.get("best_val_acc")
        min_val_loss = ckpt.get("min_val_loss")
        print(f"\n" f"Resumed from '{args.resume}' at epoch {start_epoch}. "
              f"Accuracy: {best_val_acc:.2f}%. Loss: {min_val_loss:.4f}.")

    if freeze:  # model weights are freezed
        pass
    else:  # unfreeze model weights
        for param in model.parameters():
            param.requires_grad = True

    for epoch in range(start_epoch, args.epochs):
        if float(f"{best_val_acc:.2f}") == 100.0:
            print(f"\n" f"All images classified correctly with {best_val_acc:.2f}% validation accuracy.")
            break  # exit from the training cycle

        if float(f"{min_val_loss:.4f}") == 0.0:
            print(f"\n" f"Stopping early as loss is zero {min_val_loss:.4f}.")
            break  # exit from the training cycle

        print(f"\nEpoch {epoch+1}/{args.epochs}")
        
        t0 = time.time()
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer,
                                                scaler, device, criterion)
        val_loss, val_acc = evaluate(model, val_loader, device, criterion)

        if scheduler is not None:
            scheduler.step()

        dt = time.time() - t0
        print(f"Train: loss={train_loss:.4f}, accuracy={train_acc:.2f}% | "
              f"Val: loss={val_loss:.4f}, accuracy={val_acc:.2f}% | {dt/60:.1f} min")

        # Save best
        if min_val_loss > val_loss:
            min_val_loss = val_loss
            best_val_acc = val_acc
            no_improvement_count = 0
            # torch.save(model.state_dict(), os.path.join(args.out_dir, "best_model_weights.pt"))
            # Save checkpoint
            save_checkpoint(model, optimizer, scheduler, epoch,
                            best_val_acc, min_val_loss, ckpt_path)
            print(f"\t Saved new best model: val accuracy={best_val_acc:.2f}%, "
                  f"val loss={min_val_loss:.4f}.")
        else:  # no improvement
            no_improvement_count += 1
            # Exit from training if no improvement for "patience" epochs.
            if no_improvement_count >= patience:
                print(f"\n" f"Stopping early as no improvement has been observed in {patience} epochs.")
                break  # exit from the training cycle

    print("\nTraining complete.")
    print(f"Best Val Acc: {best_val_acc:.2f}%. Minimal Val Loss: {min_val_loss:.4f}")
    print(f"Saved to: {args.out_dir}")



def create_test_loader(data_dir, batch_size, num_workers, img_size):
    _, test_tfms = build_transforms(img_size)

    test_dir = Path(data_dir) / "test"
    test_ds = datasets.ImageFolder(str(test_dir), transform=test_tfms)
    test_loader = DataLoader(
        test_ds, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, pin_memory=True
    )
    return test_loader



def main():
    # If all arguments have a default value, then adding this to the top of the notebook should be enough
    sys.argv = [""]
    
    # gpu = 3  # for DL4 server GPU can be: 0 (H800), 1 (V100), 2 (V100), 3 (V100), 4 (V100).
    # out_dir = "./outputs_ResNet50"

    # data_dir = "/share/data/lab225/ViT/img_CT_224_Gray_MF_c1_c2_c3"
    # # data_dir = "/share/data/lab225/ViT/img_6K_CT_224_Gray_RG"  # dataset directory
    
    ckpt_path = os.path.join(out_dir, "best_checkpoint.pt")

    # NOTE: all input arguments are hardcoded, i.e. have default value
    parser = argparse.ArgumentParser(description="Fine-tune ResNet50 on your dataset")
    parser.add_argument("--data_dir", type=str, default=data_dir, help="Dataset root. Expect data/train[/val] or single folder of classes.")
    parser.add_argument("--out_dir", type=str, default=out_dir, help="Where to save checkpoints.")
    parser.add_argument("--model_name", type=str, default=model_name)
    parser.add_argument("--epochs", type=int, default=50)  # 50
    parser.add_argument("--warmup_epochs", type=int, default=1)
    parser.add_argument("--batch_size", type=int, default=32)
    parser.add_argument("--lr", type=float, default=5e-5)
    parser.add_argument("--weight_decay", type=float, default=0.05)
    parser.add_argument("--img_size", type=int, default=224)
    parser.add_argument("--num_workers", type=int, default=math.ceil(multiprocessing.cpu_count()*2/3))
    parser.add_argument("--seed", type=int, default=42)
    parser.add_argument("--freeze_backbone", action="store_true", help="Only train the classification head.")
    parser.add_argument("--val_split", type=float, default=0.1, help="If no val folder, split this fraction from train.")
    parser.add_argument("--resume", type=str, default=ckpt_path, help="Path to a checkpoint to resume from.")
    args = parser.parse_args()

    set_seed(args.seed)
    os.makedirs(args.out_dir, exist_ok=True)

    device = f"cuda:{gpu}" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")
    
    train_loader, val_loader, num_classes, class_names = create_dataloaders(
        args.data_dir, args.batch_size, args.num_workers, args.img_size, args.val_split
    )
    print(f"Classes ({num_classes}): {class_names}")

    model = build_model(args.model_name, num_classes, pretrained=True,
                        freeze_backbone=args.freeze_backbone)
    model.to(device)
    
    train(model, device, args, train_loader, val_loader, num_classes,
          class_names, out_dir, ckpt_path, patience, freeze=True)

    # print(f"\n\n-----------\n" f"Unfreeze model weights and train it again")
    # train(model, device, args, train_loader, val_loader, num_classes,
    #       class_names, out_dir, ckpt_path, patience=25, freeze=False)

    test_dir = Path(args.data_dir) / "test"
    if os.path.exists(test_dir):
        test_loader = create_test_loader(
            args.data_dir, args.batch_size, args.num_workers, args.img_size
        )
        criterion = nn.CrossEntropyLoss()
        test_loss, test_acc = evaluate(model, test_loader, device, criterion)
        print(f"Test: loss={test_loss:.4f}, accuracy={test_acc:.2f}%")


if __name__ == "__main__":
    main()

Using device: cuda:3
Classes (2): ['norm', 'path']

Epoch 1/50
Train: loss=0.6994, accuracy=47.74% | Val: loss=0.6873, accuracy=54.55% | 0.5 min
	 Saved new best model: val accuracy=54.55%, val loss=0.6873.

Epoch 2/50
Train: loss=0.6890, accuracy=52.95% | Val: loss=0.6889, accuracy=54.55% | 0.2 min

Epoch 3/50
Train: loss=0.6554, accuracy=70.31% | Val: loss=0.6865, accuracy=50.00% | 0.2 min
	 Saved new best model: val accuracy=50.00%, val loss=0.6865.

Epoch 4/50
Train: loss=0.5998, accuracy=86.63% | Val: loss=0.6747, accuracy=51.52% | 0.2 min
	 Saved new best model: val accuracy=51.52%, val loss=0.6747.

Epoch 5/50
Train: loss=0.5304, accuracy=92.71% | Val: loss=0.6073, accuracy=77.27% | 0.2 min
	 Saved new best model: val accuracy=77.27%, val loss=0.6073.

Epoch 6/50
Train: loss=0.4434, accuracy=95.83% | Val: loss=0.5615, accuracy=75.76% | 0.1 min
	 Saved new best model: val accuracy=75.76%, val loss=0.5615.

Epoch 7/50
Train: loss=0.3447, accuracy=97.05% | Val: loss=0.5053, accurac

## Run Log

### Date: 02.09.2025

#### Run 1

    Best Val Acc: 87.88%. Minimal Val Loss: 0.3622
    Epoch 14/50
    Train: loss=0.0189, accuracy=100.00% | Val: loss=0.3622, accuracy=87.88% | 0.1 min
    	 Saved new best model: val accuracy=87.88%, val loss=0.3622.

### Date: 01.09.2025

#### Run 1
Run for dataset `xray_dataset` with x-rays.

Test dataset == train dataset.

    Test: loss=0.0456, accuracy=98.79%
    Best Val Acc: 87.88%. Minimal Val Loss: 0.3694
    Epoch 14/50
    Train: loss=0.0190, accuracy=100.00% | Val: loss=0.3694, accuracy=87.88% | 0.1 min
    	 Saved new best model: val accuracy=87.88%, val loss=0.3694.

### Date: 25.08.2025

#### Run 3
Final run. Dataset: `/share/data/lab225/ViT/img_CT_224_Gray_MF_c1_c2_c3`.

    Test: loss=0.0149, accuracy=99.56%
    Best Val Acc: 100.00%. Minimal Val Loss: 0.0036
    Epoch 23/50
    	[batch 50/143] loss=0.0053 acc1=100.00%
    	[batch 100/143] loss=0.0056 acc1=100.00%
    Train: loss=0.0052, accuracy=100.00% | Val: loss=0.0036, accuracy=100.00% | 1.1 min
    	 Saved new best model (val accuracy=100.00%).

#### Run 4
Final run. Dataset: `/share/data/lab225/ViT/img_6K_CT_224_Gray_RG`.

    Test: loss=0.0126, accuracy=100.00%
    Best Val Acc: 100.00%. Minimal Val Loss: 0.0125
    Epoch 11/50
    	[batch 50/281] loss=0.0259 acc1=99.75%
    	[batch 100/281] loss=0.0251 acc1=99.78%
    	[batch 150/281] loss=0.0236 acc1=99.83%
    	[batch 200/281] loss=0.0230 acc1=99.84%
    	[batch 250/281] loss=0.0217 acc1=99.85%
    Train: loss=0.0209, accuracy=99.86% | Val: loss=0.0125, accuracy=100.00% | 1.7 min
    	 Saved new best model: val accuracy=100.00%, val loss=0.0125.

#### Run 1
Run ResNet50 on CT slices. 6 classes: female-male, shoulders-lungs-abdomen.

Dataset: `/share/data/lab225/ViT/img_CT_224_Gray_MF_c1_c2_c3`

    Best Val Acc: 100.00%. Minimal Val Loss: 0.0036
    Epoch 23/50
    	[batch 50/143] loss=0.0054 acc1=100.00%
    	[batch 100/143] loss=0.0057 acc1=100.00%
    Train: loss=0.0053, acc1=100.00% | Val: loss=0.0036, acc1=100.00% | 0.5 min
    	 Saved new best model (val acc1=100.00%).

#### Run 2
Run ResNet50 on X-Ray images. 2 classes: real-generated lungs.

Dataset: `/share/data/lab225/ViT/img_6K_CT_224_Gray_RG`.

    Best Val Acc: 100.00%. Minimal Val Loss: 0.0031
    Epoch 16/50
    	[batch 50/281] loss=0.0043 acc1=100.00%
    	[batch 100/281] loss=0.0044 acc1=100.00%
    	[batch 150/281] loss=0.0039 acc1=100.00%
    	[batch 200/281] loss=0.0037 acc1=100.00%
    	[batch 250/281] loss=0.0034 acc1=100.00%
    Train: loss=0.0034, acc1=100.00% | Val: loss=0.0031, acc1=100.00% | 0.9 min
    	 Saved new best model (val acc1=100.00%).


## Part 2. Create CSV files with features

Open previously saved fine-tuned model and use it to create CVS files with features and corresponding file names.

In [3]:
import os

# === Config ===

# gpu = 3  # for DL4 server GPU can be: 0 (H800), 1 (V100), 2 (V100), 3 (V100), 4 (V100).
# out_dir = "./outputs_ResNet50"
# model_name = "wide_resnet50_2.tv2_in1k"  # ResNet50 for 224x224 images

# data_dir = "/share/data/lab225/ViT/img_CT_224_Gray_MF_c1_c2_c3"  # folder with JPG or PNG files
# # data_dir = "/share/data/lab225/ViT/img_6K_CT_224_Gray_RG"

ckpt_path = os.path.join(out_dir, "best_checkpoint.pt")
img_size = 224


import timm  # requires: pip install timm
import random
import pandas as pd

from tqdm import tqdm
from PIL import Image

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision.transforms as T


def set_seed(seed: int = 42):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = False
    cudnn.benchmark = True


def build_model(
    model_name, num_classes,
    pretrained=True, freeze_backbone=False
):
    # global_pool='avg' yields pooled features (B, C) from forward_features
    model = timm.create_model(
        model_name, pretrained=pretrained,
        num_classes=num_classes, global_pool='avg'
    )

    if freeze_backbone:
        stop_words = ['head', 'fc', 'classifier']
        for name, p in model.named_parameters():
            if all(sw not in name for sw in stop_words):
                p.requires_grad = False

    return model


# For timm Vision Transformers, 'forward_features' returns the embeddings
def extract_features(img_path):
    """ Extract features from image. """
    img = Image.open(img_path).convert('RGB')
    img_t = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        feats = model.forward_features(img_t)
        # print(feats.shape)  # torch.Size([1, 197, 768])

        # Flatten to 1D vector
        vec = model.forward_head(feats, pre_logits=True)  # get PyTorch tensor
        vec = vec.cpu().numpy()  # copy to CPU and convert PyTorch to NumPy
        # print(vec.shape)  # shape: (1, 768)
        vec = vec[0].tolist()  # squeeze NumPy and convert to list
        # Convert list to string. Left only 4 digits after the decimal points.
        vec = " ".join([f"{i:.4f}" for i in vec])
        return vec


def check_the_same_name(csv_name, csv_files, root):
    """ Check for the same name in `csv_files` list
        and change new CSV file name to make it unique. """
    check = True
    while check:
        for csv_f in csv_files:
            if csv_name in csv_f:
                # Add parent directory name to the end of CSV file.
                parent_dir = root[:-len(csv_name)-1]  # parent directory
                csv_name += "_" + os.path.basename(parent_dir)
                break  # exit from "for" loop
        check = False  # stop "while" loop
    return csv_name


def get_csv_dict(data_dir):
    """ Get CSV dictionary of directories with images. """
    csv_files = {}  # CSV file name --> path to image directory
    for root, dirs, files in os.walk(data_dir):
        # check for images in the directory
        for file_name in files:
            file_path = os.path.join(root, file_name)  # full path to the file
            # Check if the path is a file and ends with the .png extension.
            if os.path.isfile(file_path) and file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                csv_name = os.path.basename(root)
                csv_name = check_the_same_name(csv_name, csv_files, root)
                csv_files[csv_name] = root
                break  # stop checking file names
    return csv_files


def save_features(csv_name, dir_path, model, transform):
    """ Save features into a CSV file. """
    feature_list = []
    file_names = []
    f = []

    for fname in tqdm(os.listdir(dir_path), desc="Saving features", leave=False):
        if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            fpath = os.path.join(dir_path, fname)
            vec = extract_features(fpath)
            feature_list.append(vec)
            file_names.append(fname)

    for i in range(len(feature_list)):
        feature = feature_list[i]  # get the first row in (197, 768)
        f.append(feature)

    df = pd.DataFrame({"file_name": file_names, "feature_vector": f})
    csv_out_1 = os.path.join(out_dir, f"FTR_{csv_name}_fnames.csv")
    csv_out_2 = os.path.join(out_dir, f"FTR_{csv_name}_featur.csv")
    # Save file names and features in separate CSV. Don't save indices and header line
    df.to_csv(csv_out_1, columns=["file_name"], index=False, header=False)
    df.to_csv(csv_out_2, columns=["feature_vector"], index=False, header=False)
    print(f"\r" f"Features from '{dir_path}' directory are saved to file '{csv_out_2}'.")

    # # === Check ===
    # print(f"\n" f"Show contents of '{csv_out}' file")
    # print(df[:3]) # display the first 10 rows


def get_number_of_classes(path):
    """ Get number of classes. """
    directories = []
    for i in os.listdir(path):
        full_path = os.path.join(path, i)
        if os.path.isdir(full_path):
            directories.append(i)
    num_classes = len(directories)
    print(f"Classes ({num_classes}): {directories}")
    return num_classes



set_seed()  # set random seed for reproducibility
os.makedirs(out_dir, exist_ok=True)  # create output directory if doesn't exist

# === Device ===
device = f"cuda:{gpu}" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# === Load model ===
path = f"{data_dir}/train"
num_classes = get_number_of_classes(path)

model = build_model(model_name,
                    num_classes=num_classes,
                    pretrained=True,
                    freeze_backbone=True)  # load dummy model
model.to(device)
model.eval()  # switch to evaluation mode
if os.path.isfile(ckpt_path):
    ckpt = torch.load(ckpt_path, map_location="cpu")
    model.load_state_dict(ckpt["model"], strict=False)
    start_epoch = ckpt.get("epoch", 0) + 1
    best_val_acc = ckpt.get("best_val_acc")
    min_val_loss = ckpt.get("min_val_loss")
    print(f"\n" f"Resumed from '{ckpt_path}' at epoch {start_epoch}. "
          f"Accuracy: {best_val_acc:.2f}%. Loss: {min_val_loss:.4f}.\n")
else:
    print(f"\n" f"No checkpoint was found. Use standard model trained on ImageNet21k.\n")

# === Iterate through images ===
csv_files = get_csv_dict(data_dir)  # get dictionary of CSV files and dir paths

# === Preprocessing ===
transform = T.Compose([
    T.Resize(int(img_size)),
    T.ToTensor(),
    T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])

for csv_name, dir_path in csv_files.items():
    save_features(csv_name, dir_path, model, transform)

Using device: cuda:3
Classes (2): ['norm', 'path']

Resumed from './outputs_ResNet50/best_checkpoint.pt' at epoch 14. Accuracy: 87.88%. Loss: 0.3622.



Features from './xray_dataset/test/norm' directory are saved to file './outputs_ResNet50/FTR_norm_featur.csv'.


Features from './xray_dataset/test/path' directory are saved to file './outputs_ResNet50/FTR_path_featur.csv'.


Features from './xray_dataset/train/norm' directory are saved to file './outputs_ResNet50/FTR_norm_train_featur.csv'.


Features from './xray_dataset/train/path' directory are saved to file './outputs_ResNet50/FTR_path_train_featur.csv'.


## Part 3. Open CSV file with features

Open CSV file with file names.

Open another CSV file with features.

Convert features to to NumPy array (`ndarray` type).

In [4]:
import os
import numpy as np
import pandas as pd

# === Config ===

# out_dir = "./outputs_ResNet50"

# csv_out_1   = f"{out_dir}/FTR_G2_F_c1_train_fnames.csv"  # CVS file with file names
# csv_out_2   = f"{out_dir}/FTR_G2_F_c1_train_featur.csv"  # CVS file with features

# # csv_out_1   = f"{out_dir}/FTR_Generated_train_fnames.csv"  # CVS file with file names
# # csv_out_2   = f"{out_dir}/FTR_Generated_train_featur.csv"  # CVS file with features

df1 = pd.read_csv(csv_out_1, header=None)  # load the CSV file into a DataFrame, no header line
df2 = pd.read_csv(csv_out_2, header=None)

print(f"There are {len(df1)} rows in `{csv_out_1}` file.")
print(f"There are {len(df2)} rows in `{csv_out_2}` file.")

df = pd.concat([df1, df2], axis=1, ignore_index=True)  # stack two DataFrames horizontally
df.columns = ["file_name", "feature_vector"]  # rename columns from "0" and "1"
column_names = df.columns  # get the column names
print(f"Column names: {column_names}")

# stop = float("inf")  # do not stop
stop = 5  # stop after 5 rows

display(df.head(stop))  # show first 5 rows

print()
# Iterate through rows using iterrows()
for index, row in df.iterrows():
    # 'index' is the row index
    # 'row' is a Pandas Series containing the data for that row
    
    file_name = row[column_names[0]]
    print(f"{index + 1:04}.\t" f"Feature vector for file '{file_name}'")
    
    feature = row[column_names[1]]  # string
    feature = [float(x) for x in feature.split()]  # list of float numbers
    feature = np.array(feature)  # convert to NumPy format
    print(f"\t" f"  type {type(feature).__name__} has {len(feature)} of {type(feature[0]).__name__} numbers.")
    
    if index >= (stop-1):
        break

There are 336 rows in `./outputs_ResNet50/FTR_path_train_fnames.csv` file.
There are 336 rows in `./outputs_ResNet50/FTR_path_train_featur.csv` file.
Column names: Index(['file_name', 'feature_vector'], dtype='object')


,file_name,feature_vector
0,CHNCXR_0327_1_boxed.png,0.0791 0.6124 1.3399 0.1139 0.0000 0.0000 1.38...
1,CHNCXR_0328_1_boxed.png,0.1864 0.5191 0.0777 0.8289 0.5012 0.8916 0.47...
2,CHNCXR_0329_1_boxed.png,0.0000 0.2007 0.5140 0.4132 0.2312 0.0619 0.21...
3,CHNCXR_0330_1_boxed.png,0.0401 0.3747 0.2440 0.1624 0.6557 0.3950 0.48...
4,CHNCXR_0331_1_boxed.png,0.0626 0.1962 0.0507 0.4574 0.0178 0.4053 0.33...



0001.	Feature vector for file 'CHNCXR_0327_1_boxed.png'
	  type ndarray has 2048 of float64 numbers.
0002.	Feature vector for file 'CHNCXR_0328_1_boxed.png'
	  type ndarray has 2048 of float64 numbers.
0003.	Feature vector for file 'CHNCXR_0329_1_boxed.png'
	  type ndarray has 2048 of float64 numbers.
0004.	Feature vector for file 'CHNCXR_0330_1_boxed.png'
	  type ndarray has 2048 of float64 numbers.
0005.	Feature vector for file 'CHNCXR_0331_1_boxed.png'
	  type ndarray has 2048 of float64 numbers.
